# KNN - Defunciones Fetales

En este Jupyter Notebook, se busca crear un modelo de Defunciones Fetales para poder predecir el tipo de defunción fetal que se puede presentar en un embarazo, con base en las características de la madre y del embarazo.

----------
### 1. Importar librerías

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_classif
from prettytable import PrettyTable
import itertools
from sklearn.inspection import permutation_importance

-----------
### 2. Cargar datos
Como primer paso, se cargan los datos ya explorados y limpios. Estos datos se encuentran en el archivo "defunciones_clean.csv".

Las variables presentes son:

    'Área_Geográfica',
    'Asistencia_Recibida',
    'Año_Ocurrencia',
    'Año_Registro',
    'Causa_Defunción',
    'Clase_Parto',
    'Departamento_Ocurrencia',
    'Departamento_Registro',
    'Día_Ocurrencia',
    'Edad_Madre',
    'Estado_Civil_Madre',
    'Escolaridad_Madre',
    'Mes_Ocurrencia',
    'Mes_Registro',
    'Municipio_Ocurrencia',
    'Municipio_Registro',
    'Municipio_Residencia_Madre',
    'Ocupación_Madre',
    'Semanas_Gestacion',
    'Sexo',
    'Sitio_Ocurrencia',
    'Tipo_Atencion',
    'Total_Hijos_Nacidos',
    'Total_Hijos_Nacidos_Muertos',
    'Total_Hijos_Vivos',
    'Via_Parto',
    'Nacionalidad_Madre',
    'Grupo_Etnico_Madre',

In [62]:
# Cargar datos
defunciones = pd.read_csv('defunciones_clean.csv').round(0)

# Eliminar columnas no relevantes (segun analisis exploratorio anteriormente realizado)
defunciones.drop('Municipio_Residencia_Madre', axis=1, inplace=True)

defunciones.drop('Municipio_Registro', axis=1, inplace=True)

defunciones.drop('Municipio_Ocurrencia', axis=1, inplace=True)          

defunciones.drop('Mes_Registro', axis=1, inplace=True)
defunciones.drop('Día_Ocurrencia', axis=1, inplace=True)
defunciones.drop('Mes_Ocurrencia', axis=1, inplace=True)

# Aproximar todos los valores de edad a un entero


defunciones.head()

,Área_Geográfica,Asistencia_Recibida,Año_Ocurrencia,Año_Registro,Causa_Defunción,Clase_Parto,Departamento_Ocurrencia,Departamento_Registro,Departamento_Residencia_Madre,Edad_Madre,...,Semanas_Gestacion,Sexo,Sitio_Ocurrencia,Tipo_Atencion,Total_Hijos_Nacidos,Total_Hijos_Nacidos_Muertos,Total_Hijos_Vivos,Via_Parto,Nacionalidad_Madre,Grupo_Etnico_Madre
0,99,6,2009,2009,0,1,7,7,7,18.0,...,34.0,1,2,1,1.0,2.0,1.0,99,1,4
1,99,1,2009,2009,0,1,13,13,13,36.0,...,34.0,1,9,1,1.0,3.0,2.0,99,1,4
2,99,2,2009,2009,0,1,2,2,2,22.0,...,34.0,1,2,1,1.0,3.0,2.0,99,1,4
3,99,2,2009,2009,0,1,5,5,5,26.0,...,34.0,1,2,1,1.0,0.0,0.0,99,1,4
4,99,2,2009,2009,0,1,7,7,7,43.0,...,34.0,2,2,1,1.0,10.0,6.0,99,1,4


----------
### 3. Selección de variables a utilizar en modelo

En este caso, se busca determinar (a través de SelectKBest y Chi2) las mejores variables para utilizar en el modelo a realizar. Especificamente, se optó por utilizar las 20 variables que tienen una mayor correlación con la variable objetivo (Causa_Defunción).

El código mostrado a continuación selecciona los mejores features del conjunto de datos utilizando la prueba chi-cuadrado (chi-squared test). En concreto, se utiliza la función "SelectKBest" para seleccionar el número "k" de features que se especifican (en este caso, solo el mejor feature) que tienen una mayor relación con la variable objetivo. 

> En otras palabras, se analiza la correlación entre las variables y la variable objetivo, y se seleccionan las variables que tienen una mayor correlación.

In [63]:
# Split the dataset into features (X) and target variable (y)
X = defunciones.drop('Causa_Defunción', axis=1)
y = defunciones['Causa_Defunción']

# Apply feature selection using the chi-squared test
selector = SelectKBest(chi2, k=1)
X_new = selector.fit_transform(X, y)

# Order X_new by feature importance
order = np.argsort(selector.scores_)[::-1]

# Create a list of sorted feature names
feature_names = []
for i in order:
    feature_names.append(X.columns[i])

    if len(feature_names) == 20:
        break

----------
### 4. Creación del modelo KNN

A continuación se genera el modelo KNN con las variables seleccionadas que tienen mayor correlación con la variable objetivo. Asimismo, se presenta el accuracy del modelo; el cual es 0.89. Esto indica que el modelo es capaz de predecir 89 de 100 veces correctamente el tipo de muerte fetal.

In [64]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.3, random_state=42)

# Create the KNN model
knn = KNeighborsClassifier(n_neighbors=5)

# Fit the model to the training data
knn.fit(X_train, y_train)

# Predict on the test set
y_pred = knn.predict(X_test)

# Accuracy del Modelo
print(knn.score(X_test, y_test))

0.899236641221374


----------
### 5. Proceso Investigativo

En base al modelo generado, se pueden ir alterando ciertos datos para ver cómo se comporta el modelo. Por ejemplo, se puede cambiar la edad de la madre, la escolaridad, el estado civil, etc. y ver cómo se comporta el modelo. Esto con el fin de poder determinar qué características son las que más afectan el resultado de la predicción.


En este caso, se estará alterando específicamente valores relacionados con los top 10 features más importantes del modelo generado. Según el codigo presentado a continuación, estas variables son aquellas que más influyen en el resultado de la predicción.

In [65]:
# Obtener la importancia de features
importances, pvalues = f_classif(X, y)

# Imprimir la importancia de cada feature por orden
list_to_order = []
for feature, importance in zip(feature_names, importances):
    list_to_order.append((feature, importance))

list_to_order.sort(key=lambda x: x[1], reverse=True)


x, j = PrettyTable(), []
x.field_names = ["Feature", "Importance"]
for i in list_to_order:
    x.add_row(i)
    j.append(i[0])
    if len(j) == 10:
        break

print(x)

+-----------------------+--------------------+
|        Feature        |     Importance     |
+-----------------------+--------------------+
|     Tipo_Atencion     | 491.3484035614248  |
|    Área_Geográfica    |  486.24504784251   |
|  Asistencia_Recibida  | 237.02431526052104 |
|      Año_Registro     | 162.5494004171015  |
|    Sitio_Ocurrencia   | 129.83516600755243 |
|      Clase_Parto      | 102.56809844940216 |
|    Ocupación_Madre    | 101.44558358747797 |
|       Edad_Madre      | 101.07463829669375 |
|   Grupo_Etnico_Madre  | 91.06812762977205  |
| Departamento_Registro | 72.57625343252587  |
+-----------------------+--------------------+


In [67]:
defunciones_0 = defunciones[defunciones['Causa_Defunción'] == 0]
most_common_values = {}

for i in range(5, len(j) + 1):
    j_combinations = itertools.combinations(j, i)

    for j_combination in j_combinations:

        random_skip = np.random.randint(1, 3)
        if random_skip == 1:
            continue

        grouped = defunciones_0.groupby(list(j_combination)).size().reset_index(name='count')

        for index, row in grouped.iterrows():
            for col in j_combination:

                if col not in most_common_values:
                    most_common_values[col] = [row[col]]

                else:
                    most_common_values[col].append(row[col])

# Moda de cada feature
for key, value in most_common_values.items():
    most_common_values[key] = max(set(value), key=value.count)

# Imprimir la moda de cada feature
x, j = PrettyTable(), []
x.field_names = ["Feature", "Most Common Value"]
for key, value in most_common_values.items():
    x.add_row([key, value])
    j.append(key)

print(x)

In [ ]:
defunciones_1 = defunciones[defunciones['Causa_Defunción'] == 1]
most_common_values = {}

for i in range(5, len(j) + 1):
    j_combinations = itertools.combinations(j, i)

    for j_combination in j_combinations:

        random_skip = np.random.randint(1, 3)
        if random_skip == 1:
            continue

        grouped = defunciones_1.groupby(list(j_combination)).size().reset_index(name='count')

        for index, row in grouped.iterrows():
            for col in j_combination:

                if col not in most_common_values:
                    most_common_values[col] = [row[col]]

                else:
                    most_common_values[col].append(row[col])

# Moda de cada feature
for key, value in most_common_values.items():
    most_common_values[key] = max(set(value), key=value.count)

# Imprimir la moda de cada feature
x, j = PrettyTable(), []
x.field_names = ["Feature", "Most Common Value"]
for key, value in most_common_values.items():
    x.add_row([key, value])
    j.append(key)

print(x)

+---------+-------------------+
| Feature | Most Common Value |
+---------+-------------------+
+---------+-------------------+
